# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF


In [427]:
import pandas as pd
import re
from data_utils import *

In [428]:
names_data = pd.read_csv('dataframes/names_gpw.csv')

In [429]:
def clean_column_names(columns):
    cleaned_columns = []
    for column in columns:
        column = column.lower()
        column = re.sub(r'\(.*?\)', '', column)
        column = re.sub(r'[^\w\s]', '', column)
        column = re.sub(r'\s+', '_', column)
        column = column.rstrip('_')
        cleaned_columns.append(column)
    return cleaned_columns

In [430]:
def count_rows_by_filename(df):
    counts = df.groupby('filename').size().reset_index(name='count')
    return counts

# FULL DF CLEARNING

In [431]:
full_data = pd.read_csv("dataframes/dataframe_gpw.csv")

In [432]:
full_merged_df = pd.merge(full_data, names_data, on='filename')

In [433]:
full_merged_df = full_merged_df.sort_values(by=['filename', 'End of period'])

In [434]:
full_merged_df.columns = clean_column_names(full_merged_df.columns)

In [435]:
count_rows_by_filename(full_merged_df)

,filename,count
0,11BIT.xlsx,49
1,3LPSA.xlsx,6
2,3RGAMES.xlsx,48
3,AB.xlsx,70
4,AC.xlsx,51
...,...,...
332,ZPUE.xlsx,95
333,ZREMBCH.xlsx,65
334,ZUE.xlsx,53
335,ZUK.xlsx,67


In [436]:
# Delete all companies under 4 because its not enough to cover any change in assets or liabilities
filenames_to_remove = [
    'SADOVAYA.xlsx',
    'BMW.xlsx',
    'KERNEL.xlsx',
    'SILVANO.xlsx',
    'IMCSA.xlsx',
    'STSHOLDING.xlsx'
]

df_filtered = full_merged_df[~full_merged_df['filename'].isin(filenames_to_remove)]

## Missing data

In [437]:
full_merged_df.isnull().sum()

end_of_period                                                      0
revenues_from_sales                                              945
operating_profitloss                                             737
net_profitloss_attributable_to_equity_holders_of_the_parent      577
total_comprehensive_income_attributable_to_equity_holders      17723
                                                               ...  
current_financial_liabilities                                  19906
noncurrent_financial_liabilities                               19906
financial_liabilities                                          19906
company_name                                                       0
sector                                                            13
Length: 108, dtype: int64

In [438]:
# Sektor NaN
rows_with_nan_in_sektor = full_merged_df[full_merged_df['sector'].isna()]
rows_with_nan_in_sektor

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,aktywa,...,debt_service_ratio,rate_debt_security,leverage,asset_utilization,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,financial_liabilities,company_name,sector
49,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.301800e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
50,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053760e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
51,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613490e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
52,2020-12-31,34515.00000,3929.00000,138.00000,138.0,4114.00000,8513.00000,-3261.00000,-5264.0000,1.640630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
53,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.547040e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
54,2021-12-31,39704.00000,3898.00000,2169.00000,2169.0,4459.00000,5831.00000,-2718.00000,-3195.0000,1.571200e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN
2522,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166580e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN
2523,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.295270e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN
18362,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.065330e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN
18363,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.348397e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN


# Notes
Manually checking for each company

3LP SA - handel
BMW AG - motoryzacja
Uf Games SA - gry


In [439]:
company_to_sector = {
    '3LP SA': 'handel',
    'BMW AG': 'motoryzacja',
    'Uf Games SA': 'gry'
}

full_merged_df['sector'] = full_merged_df.apply(lambda row: company_to_sector[row['company_name']] if pd.isna(row['sector']) else row['sector'], axis=1)

In [440]:
# Rest of financial data - fill NaN with 0 (= no change)
full_merged_df = full_merged_df.fillna(0)

In [441]:
full_merged_df.columns.tolist()

['end_of_period',
 'revenues_from_sales',
 'operating_profitloss',
 'net_profitloss_attributable_to_equity_holders_of_the_parent',
 'total_comprehensive_income_attributable_to_equity_holders',
 'depreciation',
 'cash_flow_from_operating_activities',
 'cash_flow_from_investing_activities',
 'cash_flow_from_financing_activities',
 'aktywa',
 'noncurrent_assets',
 'current_assets',
 'equity_shareholders_of_the_parent',
 'noncurrent_liabilities',
 'current_liabilities',
 'number_of_shares',
 'earnings_per_share',
 'total_comprehensive_income_per_share',
 'book_value_per_share',
 'accounting_standard',
 'filename',
 'property_plant_and_equipment',
 'exploration_for_and_evaluation_of_mineral_resources',
 'intangible_assets',
 'goodwill',
 'investment_property',
 'rightofuse_assets',
 'investment_in_affiliates',
 'noncurrent_financial_assets',
 'noncurrent_loans_and_receivables',
 'deferred_income_tax',
 'noncurrent_deferred_charges_and_accruals',
 'noncurrent_derivative_instruments',
 'other

# Change each indicator to % value
return_on_equity
return_on_assets
leverage
asset_utilization
debt_ratio
debt_service_ratio
rate_debt_security
working_capital_ratio

In [442]:
columns_to_convert = [
    'return_on_equity',
    'return_on_assets',
    'leverage',
    'asset_utilization',
    'debt_ratio',
    'debt_service_ratio',
    'rate_debt_security',
    'working_capital_ratio'
]

# Konwersja wartości w kolumnach na format dziesiętny
full_merged_df[columns_to_convert] = full_merged_df[columns_to_convert] / 100

In [443]:
column_mapping = {
    'financial_liabilities': 'total_liability',
    'aktywa': 'total_assets'
}

full_merged_df = full_merged_df.rename(columns=column_mapping)

In [447]:
full_merged_df['company_name'].unique()

array(['11 bit studios SA', '3LP SA', '3R Games SA', 'AB SA', 'AC SA',
       'Action SA', 'Adiuvo Investments SA', 'Agora SA', 'Agroton plc.',
       'AUGA Group AB', 'All in! Games SA', 'Ailleron SA',
       'Airway Medix SA', 'Alumetal SA', 'Ambra SA', 'Amica SA',
       'Ampli SA w upadłości układowej', 'AmRest Holdings SE',
       'Answear.com SA', 'Apator SA', 'Aplisens SA', 'APS Energia SA',
       'Arctic Paper SA', 'Arteria SA', 'Artifex Mundi SA',
       'ASBISc Enterprises plc.', 'ASM Group SA', 'Asseco BS SA',
       'Asseco Poland SA', 'Asseco SEE SA', 'Atende SA',
       'Atlanta Poland SA', 'ATM Grupa SA', 'Atrem SA', 'Auto Partner SA',
       'Grupa Azoty SA', 'British Automotive Holding SA',
       'Elektrociepłownia Będzin SA', 'Benefit Systems SA', 'Berling SA',
       'Betacom SA', 'Big Cheese Studio SA', 'Bioceltix SA',
       'BioMaxima SA', 'Biomed Lublin SA', 'Bio Planet SA', 'Bioton SA',
       'BMW AG', 'LW Bogdanka SA', 'BoomBit SA', 'Boryszew SA',
       'Bo

## Feature Engineering

In [445]:
# Change in capital or assets structure
# feature: change in last quarter for each company (every major data attribute)
# feature: change in last two quarters for each company (every major data attribute)
# feature: change in last four quarters for each company (every major data attribute)

In [ ]:
full_merged_df

# NN DATA CLEANING (to póki co nieistotne - trzeba skończyć pelny plik, a potem wybrać sobie odpowiednie kolumny)

In [ ]:
columns_to_keep = [
    'filename',
    # Additional Information
    'Current financial liabilities',
    'Non-current financial liabilities',
    'Financial liabilities (total)',
    
    # DuPont Indicators
    'Return on equity (ROE)',
    'Return on assets (ROA)',
    'Leverage (EM)',
    'Net profit margin',
    'Asset utilization (AU)',

    # Indicators
    'Current ratio',
    'Quick ratio',
    'Cash ratio',
    'Receivables turnover',
    'Inventory turnover',
    'The operating cycle',
    'Rotation commitments',
    'Cash conversion cycle',
    'Rotation assets',
    'Rotation of assets',
    'Assets ratio',
    'Debt ratio',
    'Debt service ratio',
    'Rate debt security',
    
    # Basic Information
    'End of period',
    
    # Aktywa (Assets)
    'Non-current assets',
    'Current assets',
    
    # Pasywa (Liabilities)
    'Equity shareholders of the parent',
    'Non-current liabilities',
    'Current liabilities'
]

In [ ]:
nn_data = pd.read_csv("dataframes/nn_gpw_data.csv")

In [ ]:
nn_merged_df = pd.merge(nn_data, names_data, on='filename')

In [ ]:
nn_merged_df = nn_merged_df.sort_values(by=['filename', 'End of period'])

In [ ]:
nn_merged_df.columns = clean_column_names(nn_merged_df.columns)

# MINI DATAFRAME